In [1]:
import argparse, json, os
import numpy as np
import h5py
import codecs

/home/paperspace/anaconda3/envs/music/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# -*- coding: utf-8 -*-
def preprocess(input_txt, output_h5, output_json, val_frac=0.1, test_frac=0.1, encoding='utf-8'):
    # First go the file once to see how big it is and to build the vocab
    token_to_idx = {}
    total_size = 0
    with codecs.open(input_txt, 'r', encoding) as f:
        for line in f:
            total_size += len(line)
            for char in line:
                if char not in token_to_idx:
                    token_to_idx[char] = len(token_to_idx) + 1

    # Now we can figure out the split sizes
    val_size = int(val_frac * total_size)
    test_size = int(test_frac * total_size)
    train_size = total_size - val_size - test_size

    print('Total vocabulary size: %d' % len(token_to_idx))
    print('Total tokens in file: %d' % total_size)
    print('  Training size: %d' % train_size)
    print('  Val size: %d' % val_size)
    print('  Test size: %d' % test_size)

    # Choose the datatype based on the vocabulary size
    dtype = np.uint8
    if len(token_to_idx) > 255:
        dtype = np.uint32
        
    print('Using dtype ', dtype)

    # Just load data into memory ... we'll have to do something more clever
    # for huge datasets but this should be fine for now
    train = np.zeros(train_size, dtype=dtype)
    val = np.zeros(val_size, dtype=dtype)
    test = np.zeros(test_size, dtype=dtype)
    splits = [train, val, test]

    # Go through the file again and write data to numpy arrays
    split_idx, cur_idx = 0, 0
    with codecs.open(input_txt, 'r', encoding) as f:
        for line in f:
            for char in line:
                splits[split_idx][cur_idx] = token_to_idx[char]
                cur_idx += 1
                if cur_idx == splits[split_idx].size:
                    split_idx += 1
                    cur_idx = 0

    # Write data to HDF5 file
    with h5py.File(output_h5, 'w') as f:
        f.create_dataset('train', data=train)
        f.create_dataset('val', data=val)
        f.create_dataset('test', data=test)

    # For 'bytes' encoding, replace non-ascii characters so the json dump
    # doesn't crash
    if encoding is None:
        new_token_to_idx = {}
        for token, idx in token_to_idx.items():
            if ord(token) > 127:
                new_token_to_idx['[%d]' % ord(token)] = idx
            else:
                new_token_to_idx[token] = idx
        token_to_idx = new_token_to_idx

    # Dump a JSON file for the vocab
    json_data = {
        'token_to_idx': token_to_idx,
        'idx_to_token': {v: k for k, v in token_to_idx.items()},
    }
    with open(output_json, 'w') as f:
        json.dump(json_data, f)


In [5]:
from constants import *

In [6]:
preprocess(f'{CONCAT_DIR}/concat_corpus.utf', f'{CONCAT_DIR}/concat_corpus.h5', f'{CONCAT_DIR}/concat_corpus.json')

Total vocabulary size: 108
Total tokens in file: 341705
  Training size: 273365
  Val size: 34170
  Test size: 34170
Using dtype  <class 'numpy.uint8'>
